<a href="https://colab.research.google.com/github/mille055/AIPI540_individual_project/blob/main/notebooks/AIPI540_IP_fusion_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!git clone 'https://github.com/mille055/AIPI540_individual_project.git'

Cloning into 'AIPI540_individual_project'...
remote: Enumerating objects: 305, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 305 (delta 4), reused 7 (delta 4), pack-reused 291
Receiving objects: 100% (305/305), 91.87 MiB | 17.71 MiB/s, done.
Resolving deltas: 100% (140/140), done.
Updating files: 100% (35/35), done.


In [2]:
!pip install -r https://raw.githubusercontent.com/mille055/AIPI540_individual_project/main/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=0bb81d115acf853f49cb9384daedcc74db99f534239c27dd770ea7074e00670a
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully bu

In [4]:
!pip install pydicom==2.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.5 MB/s eta 0:00:00
  Attempting uninstall: pydicom
    Found existing installation: pydicom 2.3.1
    Uninstalling pydicom-2.3.1:
      Successfully uninstalled pydicom-2.3.1


In [5]:
!pip show pydicom

Name: pydicom
Version: 2.1.2
Summary: Pure python package for DICOM medical file reading and writing
Home-page: https://github.com/pydicom/pydicom
Author: Darcy Mason and contributors
Author-email: darcymason@gmail.com
License: MIT
Location: /usr/local/lib/python3.9/dist-packages
Requires: 
Required-by: 


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import torchvision
import pydicom
import monai
import pickle

from torchvision import datasets, models, transforms


In [7]:
#local imports
from AIPI540_individual_project.scripts.config import *
# from AIPI540_individual_project.scripts.train_pixel_model import train_model
# from AIPI540_individual_project.scripts.train_text_model import load_text_data, train_text_model, list_incorrect_text_predictions
# from AIPI540_individual_project.scripts.utils import *

# Load Datasets


In [8]:
train_datafile = '/content/AIPI540_individual_project/data/X_train02282023.pkl'
test_datafile = '/content/AIPI540_individual_project/data/X_test02282023.pkl'

In [9]:
def load_dataset(train_file, test_file):
  with open(train_file, 'rb') as f:
    train_df = pickle.load(f)
  with open(test_file, 'rb') as g:
    test_df = pickle.load(g)

  return train_df, test_df

In [10]:
train, test = load_dataset(train_datafile, test_datafile)

In [28]:
train.columns.tolist()

['fname_x',
 'StudyInstanceUID',
 'SeriesInstanceUID',
 'SeriesNumber',
 'SeriesDescription_x',
 'AcquisitionNumber',
 'InstanceNumber',
 'ImageOrientationPatient',
 'MRAcquisitionType',
 'AngioFlag',
 'SliceThickness',
 'RepetitionTime',
 'EchoTime',
 'EchoTrainLength',
 'PixelSpacing',
 'ContrastBolusAgent',
 'InversionTime',
 'seq_E',
 'seq_EP',
 'seq_G',
 'seq_IR',
 'seq_P',
 'seq_R',
 'seq_S',
 'seq_SE',
 'var_E',
 'var_K',
 'var_N',
 'var_O',
 'var_OSP',
 'var_P',
 'var_S',
 'var_SK',
 'var_SP',
 'var_SS',
 'opt_2',
 'opt_A',
 'opt_ACC_GEMS',
 'opt_D',
 'opt_EDR_GEMS',
 'opt_EPI_GEMS',
 'opt_F',
 'opt_FAST_GEMS',
 'opt_FC',
 'opt_FC_SLICE_AX_GEMS',
 'opt_FILTERED_GEMS',
 'opt_FS',
 'opt_FT_GEMS',
 'opt_I',
 'opt_IDEAL_GEMS',
 'opt_MP_GEMS',
 'opt_MRCP_GEMS',
 'opt_NPW',
 'opt_P',
 'opt_PFF',
 'opt_PFP',
 'opt_RTR_GEMS',
 'opt_S',
 'opt_SAT2',
 'opt_SAT_GEMS',
 'opt_SEQ_GEMS',
 'opt_SFS',
 'opt_SP',
 'opt_SS_GEMS',
 'opt_T',
 'opt_TRF_GEMS',
 'opt_VASCTOF_GEMS',
 'opt_W',
 'opt_X'

In [29]:
label_df = pd.read_csv('/content/AIPI540_individual_project/data/cmm_labels.txt')

In [30]:
label_df

,085,exam1,4,14
0,85,exam1,3,19
1,85,exam1,10,9
2,85,exam1,22,16
3,85,exam1,21,6
4,85,exam1,2,7
...,...,...,...,...
2209,83,exam1,20,16
2210,83,exam1,8,9
2211,83,exam1,18,16
2212,83,exam1,17,0
